<a href="https://colab.research.google.com/github/Emmanuel-Ayeni/foulMvSoccerNet-/blob/main/avar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install soccernet
# Libaries
import os
import zipfile
import json
import pandas as pd
import SoccerNet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 118.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 7.5 MB/s eta 0:00:00


In [ ]:
!ls path/to/SoccerNet/mvfouls/ # List the content of the directory

challenge.zip  test.zip  train.zip  valid.zip


In [ ]:
from SoccerNet.Downloader import SoccerNetDownloader as SNdl
mySNdl = SNdl(LocalDirectory="path/to/SoccerNet")
mySNdl.downloadDataTask(task="mvfouls", split=["train","valid","test","challenge"], password="s0cc3rn3t")

In [ ]:
!ls -la path/to/SoccerNet/mvfouls/ # Verify ZIP downloaad correctly

total 3246244
drwxr-xr-x 2 root root       4096 Feb 26 13:03 .
drwxr-xr-x 3 root root       4096 Feb 26 13:00 ..
-rw-r--r-- 1 root root  245990818 Feb 26 13:13 challenge.zip
-rw-r--r-- 1 root root  268201537 Feb 26 13:03 test.zip
-rw-r--r-- 1 root root 2459423137 Feb 26 13:03 train.zip
-rw-r--r-- 1 root root  350517090 Feb 26 13:03 valid.zip


In [ ]:
'''from google.colab import drive
def extract_zip_files(zip_dir, extract_to):
    """
    Extracts all files from a zip archive.

    Args:
        zip_path (str): The path to the zip file.
        extract_to (str): The directory to extract the files to.
    """
    if not os.path.exists(extract_to):
      os.makedirs(extract_to)

    try:
        for filename in os.listdir(zip_dir):
          if filename.endswith('.zip'):
            zip_filepath = os.path.join(zip_dir, filename)
            print(f"Extracting {filename}...")

            with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
              zip_ref.extractall(extract_to)
            print(f"Extracted {filename} to {extract_to}")

    except Exception as e:
      print(f"Error during extraction: {e}")'''


Extraction of Videos ZIP files and create directories for stoarage

In [ ]:
def extract_zip_files_to_subdirs(zip_dir, base_extract_dir):
    """
    Extracts each zip file in zip_dir to a subdirectory
    named after the zip file (without the .zip extension)
    within base_extract_dir.  No files are extracted directly
    to base_extract_dir.
    """

    try:
        for filename in os.listdir(zip_dir):
            if filename.endswith('.zip'):
                zip_filepath = os.path.join(zip_dir, filename)
                subdir_name = os.path.splitext(filename)[0]  # Name from filename
                extract_to = os.path.join(base_extract_dir, subdir_name)

                # Create the base directory if it doesn't exist.
                if not os.path.exists(base_extract_dir):
                    os.makedirs(base_extract_dir)

                if not os.path.exists(extract_to):
                    os.makedirs(extract_to)  # Create subdir if it doesn't exist.

                print(f"Extracting {filename} to {extract_to}...")

                with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
                    zip_ref.extractall(extract_to)

                print(f"Extracted {filename} to {extract_to}")

    except Exception as e:
        print(f"Error during extraction: {e}")


In [ ]:
zip_path = "./path/to/SoccerNet/mvfouls/"  # Replace with actual path
base_extract_dir = "./soccer_extract_data"  # Base directory for subfolders

extract_zip_files_to_subdirs(zip_path, base_extract_dir)

Extracting test.zip to ./soccer_extract_data/test...
Extracted test.zip to ./soccer_extract_data/test
Extracting train.zip to ./soccer_extract_data/train...
Extracted train.zip to ./soccer_extract_data/train
Extracting valid.zip to ./soccer_extract_data/valid...
Extracted valid.zip to ./soccer_extract_data/valid
Extracting challenge.zip to ./soccer_extract_data/challenge...
Extracted challenge.zip to ./soccer_extract_data/challenge


total 136
drwxr-xr-x    6 root root  4096 Feb 26 13:20 .
drwxr-xr-x    1 root root  4096 Feb 26 13:38 ..
drwxr-xr-x  275 root root 12288 Feb 26 13:20 challenge
drwxr-xr-x  303 root root 12288 Feb 26 13:19 test
drwxr-xr-x 2918 root root 86016 Feb 26 13:20 train
drwxr-xr-x  413 root root 16384 Feb 26 13:20 valid


In [ ]:
import cv2 # OpenCV for video processing

In [ ]:
!ls ./soccer_extract_data/test/


action_0    action_132	action_167  action_200	action_235  action_27	action_33  action_68
action_1    action_133	action_168  action_201	action_236  action_270	action_34  action_69
action_10   action_134	action_169  action_202	action_237  action_271	action_35  action_7
action_100  action_135	action_17   action_203	action_238  action_272	action_36  action_70
action_101  action_136	action_170  action_204	action_239  action_273	action_37  action_71
action_102  action_137	action_171  action_205	action_24   action_274	action_38  action_72
action_103  action_138	action_172  action_206	action_240  action_275	action_39  action_73
action_104  action_139	action_173  action_207	action_241  action_276	action_4   action_74
action_105  action_14	action_174  action_208	action_242  action_277	action_40  action_75
action_106  action_140	action_175  action_209	action_243  action_278	action_41  action_76
action_107  action_141	action_176  action_21	action_244  action_279	action_42  action_77
action_108  ac

In [ ]:
# Path to Video and JSON Annotation
video_path = "./soccer_extract_data/test/action_1.mp4"  # Replace with actual path
json_path = "./soccer_extract_data/test/annotations.json"  # Replace with actual path

In [ ]:
# Load JSON Annotations
with open(json_path, 'r') as f:
    annotations = json.load(f)


In [ ]:
# Get the first key of the dictionary
first_key = list(annotations.keys())[1]

# Access the value associated with the first key
first_item = annotations[first_key]

# Print the first item
print(first_item)

301


In [ ]:
# Get the first key
first_key = list(annotations.keys())[0]

# Get the first value using the first key
first_value = annotations[first_key]

# Print the first key-value pair
print((first_key, first_value))

('Set', 'test')


In [42]:
!ls -lt

total 18164
-rw-r--r-- 1 root root 18581255 Feb 26 15:33 yolov5su.pt
drwxr-xr-x 3 root root     4096 Feb 26 13:38 frames
drwxr-xr-x 6 root root     4096 Feb 26 13:20 soccer_extract_data
drwxr-xr-x 3 root root     4096 Feb 26 13:00 path
drwxr-xr-x 1 root root     4096 Feb 21 14:21 sample_data


In [ ]:
# Video Processing (Frame Extraction and Saving)
def process_video(video_path, output_dir, frame_rate=5):  # Adjust frame rate as needed
    """Extracts frames from a video and saves them as images."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    video_name = os.path.splitext(os.path.basename(video_path))[0]
    output_subdir = os.path.join(output_dir, video_name)
    if not os.path.exists(output_subdir):
        os.makedirs(output_subdir)

    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    success, frame = cap.read()

    while success:
        if frame_count % frame_rate == 0: # Extract every 'frame_rate'-th frame
            frame_filename = f"frame_{frame_count:05d}.jpg"  # Format frame number
            frame_path = os.path.join(output_subdir, frame_filename)
            cv2.imwrite(frame_path, frame)  # Save the frame

        frame_count += 1
        success, frame = cap.read() # Read the next frame
    cap.release()
    print(f"Extracted frames from {video_path} to {output_subdir}")
    return output_subdir


In [ ]:
# Accessing Annotations (Example: Foul Events)
foul_events = annotations.get('fouls', [])  # Get the list of foul events

for foul in foul_events:
    start_frame = foul.get('start_frame') # Access start and end frame of the foul
    end_frame = foul.get('end_frame')
    # ... process foul information (e.g., player IDs, foul type) ...
    print(f"Foul: Start Frame: {start_frame}, End Frame: {end_frame}")
    # Example: Accessing other information
    # player1_id = foul.get('player1_id')
    # foul_type = foul.get('type')  # If available in the JSON

In [ ]:
# Creating Labels (Example: Binary Foul/No Foul)
# This is a simplified example.  You'll likely need to adapt it
# based on how your annotations are structured and what kind of
# labels you want to create.

# Create a list to hold labels. Initialize to 0 (no foul)
num_frames = len(os.listdir(frames_directory))  # Get number of frames in the video
labels = [0] * num_frames

for foul in foul_events:
    start_frame = foul.get('start_frame')
    end_frame = foul.get('end_frame')
    if start_frame is not None and end_frame is not None:
        for i in range(start_frame, end_frame + 1):  # Mark frames as foul (1)
            if i < num_frames: # Check to avoid index error
                labels[i] = 1

# Now 'labels' is a list where each element corresponds to a frame.
# 0 means no foul, 1 means foul.
print("Labels:", labels)

Labels: []


In [31]:
!pip install torch torchvision torchaudio ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.9/921.9 kB 44.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [32]:
import cv2
import torch
from ultralytics import YOLO
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression # Example model
from sklearn.metrics import classification_report, accuracy_score

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [33]:
# --- Data Loading and Preprocessing ---
def process_video(video_path, json_path, output_dir, frame_rate=5):
    """Processes a single video and its annotations."""
    # ... (same video processing as before) ...
    with open(json_path, 'r') as f:
        annotations = json.load(f)

    # Multi-class Label Creation (Example - adapt to your foul types)
    num_frames = len(os.listdir(os.path.join(output_dir, os.path.splitext(os.path.basename(video_path))[0])))
    labels = [0] * num_frames  # 0: No foul, 1: Foul Type 1, 2: Foul Type 2, ...
    foul_events = annotations.get('fouls', [])
    for foul in foul_events:
        start_frame = foul.get('start_frame')
        end_frame = foul.get('end_frame')
        foul_type = foul.get('type')  # Assuming 'type' in your JSON (e.g., 1, 2, 3)
        if start_frame is not None and end_frame is not None and foul_type is not None:
            for i in range(start_frame, end_frame + 1):
                if i < num_frames:
                    labels[i] = foul_type  # Assign the correct foul type

    return output_dir, labels

In [34]:
# --- Object Detection and Tracking (YOLOv5) ---
model = YOLO('yolov5s.pt')  # Load a pre-trained YOLOv5s model (small size)

def detect_and_track(image_path):
    """Performs object detection and tracking on an image."""
    results = model(image_path)  # Run object detection
    detections = []
    for *xyxy, conf, cls in results[0].boxes.data:  # Iterate through detections
        x1, y1, x2, y2 = map(int, xyxy)  # Convert bounding box to integers
        class_id = int(cls)
        if class_id == 0:  # Assuming 0 is the class ID for "person" (player)
            detections.append([x1, y1, x2, y2, conf])  # Add bounding box and confidence
        elif class_id == 1: # Assuming 1 is the class ID for "ball"
            ball_detections.append([x1, y1, x2, y2, conf])

    return detections, ball_detections  # Return detections (players)

PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



100%|██████████| 17.7M/17.7M [00:00<00:00, 28.2MB/s]


In [35]:
# --- Feature Extraction (Example - Adapt as needed) ---
def extract_features(image_path, detections):
    """Extracts features from an image given detections."""
    img = cv2.imread(image_path)
    features = []
    for x1, y1, x2, y2, conf in detections:
        # Example: Crop the detected player and extract some simple features
        player_img = img[y1:y2, x1:x2]
        if player_img.size > 0: # Check if the image is valid
            mean_color = np.mean(player_img, axis=(0, 1))  # Average color
            features.append(mean_color)
    return np.array(features)